In [ ]:
# Вариант № 4
## Используя метод свертки анализа замкнутых экспоненциальных сетей массового обслуживания, определить стационарные характеристики сети N с заданными параметрами.

In [3]:
import numpy as np

In [4]:
# Решение системы omega*theta=omega с условием нормировки sum(omega)=1
def get_omega(L, theta):
    omega = np.array([1/(L) for _ in range(L)]) 
    for _ in range(1000000):
        n_omega = omega @ theta
        omega = n_omega
    return omega

In [5]:
# Функция для расчета нормализующих констант
def get_g(L, Q, x):
    g = np.zeros((Q+1, L))
    for i in range(Q + 1):
        g[i][0] = x[0]**i
    g[0] = [1] * L    
    for i in range(1, Q + 1):
        for j in range(1, L):
            g[i][j] = g[i][j-1] + x[j] * g[i-1][j]
    return g

In [6]:
# Условия задачи
L = 7 # число систем обслуживания в сети
Q = 10 # число требований в сети
mu = np.array([1.3, 1.2, 2.5, 0.8, 1.2, 1, 0.7]) # интенсивности обслуживания требований
theta = np.array([ # маршрутная матрица
    [0,      0,       0.6,     0,       0.3,     0.1,  0],
    [0,      0,       0,       0.5,     0.5,     0,    0],
    [0,      1,       0,       0,       0,       0,    0],
    [0,      0,       0.2,     0,       0.5,     0,    0.3],
    [0.2,    0.6,     0,       0,       0,       0,    0.2],
    [0.7,    0,       0,       0.1,     0,       0,    0.2],
    [0,      0,       1,       0,       0,       0,    0]
])

In [7]:
# Решение уравнений omega*theta = omega с условием нормировки
omega = get_omega(L, theta)
x = omega/mu

In [8]:
# Вычисление нормализующей константы G(Q,L) и значения величин g(Y,L), Y = 1,..,Q 
g = get_g(L, Q, x)

In [9]:
# Создание массивов для хранения стационарных характеристик
## Вероятности что в системах m и более требований
P = np.zeros((Q+1, L))
## Вероятности что в системах ровно m требований
p = np.zeros((Q+1, L))
## М.о. числа требований в системах
s = np.zeros(L)
## М.о. числа занятых приборов в системах
h = np.zeros(L)
## Интенсивности входного потока требований в системах
lambdas = np.zeros(L)
## М.о. длительности пребывания требований в системах
u = np.zeros(L)
## Коэффициенты использования систем
psi = np.zeros(L)

In [10]:
# Вычисление характеристик систем сети (по формулам 1.2)
for i in range(Q + 1):
    for j in range(L):
        P[i][j] = x[j]**i * (g[Q-i][L-1] / g[Q][L-1])
        p[i][j] = (x[j]**i / g[Q][L-1]) * (g[Q-i][L-1] - x[j] * g[Q-i-1][L-1])

for j in range(L):
    p[10][j] = (x[j]**10 / g[Q][L-1]) * (g[Q-10][L-1] - x[j] * 0)

for i in range(L):
    for j in range(1, Q + 1):
        s[i] += x[i]**j * (g[Q-j][L-1] / g[Q][L-1])
    h[i] = x[i] * (g[Q-1][L-1] / g[Q][L-1])
    lambdas[i] = h[i] * mu[i]
    u[i] = s[i] / lambdas[i]
    psi[i] = lambdas[i] / mu[i]

In [11]:
# Вывод результатов 
print('\nСтационарные вероятности пребывания в системах m или более требований:\n')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(P[i][j], 4)), end=' | ')
    print('\n', '_' * 80)
print('=' * 120)   
        
print('\nСтационарные вероятности пребывания в системах ровно m требований:\n')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(p[i][j], 4)), end=' | ')
    print('\n', '_' * 80)
print('=' * 120)   

print('\nМ.о. числа требований в системах:\n')
for i in range(L):
    print(f'Система {i + 1}:\t{s[i]}')
print('=' * 120)

print('\nМ.о. длительности пребывания требований в системах:\n')
for i in range(L):
    print(f'Система {i + 1}:\t{u[i]}')
print('=' * 120)

print('\nИнтенсивности потоков требований в системах:\n')
for i in range(L):
    print(f'Система {i + 1}:\t{lambdas[i]}')
print('=' * 120)

print('\nКоэффициенты использования систем:\n')
for i in range(L):
    print(f'Система {i + 1}:\t{psi[i]}')
print('=' * 120)


Стационарные вероятности пребывания в системах m или более требований:

0	| 00001.0 | 00001.0 | 00001.0 | 00001.0 | 00001.0 | 00001.0 | 00001.0 | 
 ________________________________________________________________________________
1	| 00.1462 | 00.9173 | 00.2283 | 00.6904 | 00.7363 | 000.019 | 00.4946 | 
 ________________________________________________________________________________
2	| 00.0209 | 00.8241 | 000.051 | 00.4668 | 00.5309 | 00.0004 | 00.2396 | 
 ________________________________________________________________________________
3	| 00.0029 | 00.7208 | 00.0111 | 00.3072 | 00.3727 | 00000.0 | 000.113 | 
 ________________________________________________________________________________
4	| 00.0004 | 00.6087 | 00.0023 | 00.1953 | 00.2526 | 00000.0 | 00.0514 | 
 ________________________________________________________________________________
5	| 00.0001 | 00.4905 | 00.0005 | 00.1184 | 00.1634 | 00000.0 | 00.0223 | 
 __________________________________________________________________